In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import linear_ncem_adata2data_helper
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM
from gpu_spatial_graph_pipeline.data.datasets import DatasetHartmann

In [7]:
#Mibitof
dataset = DatasetHartmann(data_path='./example_data/hartmann/')
adata = list(dataset.img_celldata.values())
feature_names=['Cluster_preprocessed','donor']

def mibitof2data(adata): 
    return linear_ncem_adata2data_helper(adata,feature_names)


num_features=(len(set(dataset.celldata.obs[feature_names[0]])),len(set(dataset.celldata.obs[feature_names[1]])))
num_genes=dataset.celldata.shape[1]


Loading data from raw files
registering celldata


/home/sel/mambaforge/envs/pyg/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates
Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.


In [8]:
dm = GraphAnnDataModule(adata=adata, adata2data_fn=mibitof2data, num_workers = 12, batch_size=10,learning_type='graphwise')
model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)

In [9]:
trainer:pl.Trainer = pl.Trainer(accelerator='gpu' if torch.torch.cuda.is_available() else 'cpu',
                                max_epochs=100,log_every_n_steps=10)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model,datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 2.8 K 
1 | model_mu    | LinearSpatial   | 2.8 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/sel/mambaforge/envs/pyg/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7f5efbf8a550>
Traceback (most recent call last):
  File "/home/sel/mambaforge/envs/pyg/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 24807, 24819, 24831) exited unexpectedly

IOStream.flush timed out


In [11]:
trainer.test(model, datamodule=dm)

Testing DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 55.65it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │   -0.11146972328424454    │
│       test_r2_score       │    -0.2056509477392449    │
└───────────────────────────┴───────────────────────────┘

[{'test_r2_score': -0.2056509477392449, 'test_loss': -0.11146972328424454}]